Paso 1: Instalación de las bibliotecas necesarias

In [1]:
!pip install nltk tensorflow

Paso 2: Importar las bibliotecas y descargar recursos

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Paso 3: Preparar los datos

In [3]:
intents = {
"intents": [
{"tag": "saludo", "patterns": ["Hola", "¿Cómo estás?", "Buenos días"], "responses": ["Hola", "¡Hola! ¿Cómo puedo ayudarte?"]},
{"tag": "despedida", "patterns": ["Adiós", "Nos vemos", "Hasta luego"], "responses": ["Adiós", "¡Hasta luego!"]},
{"tag": "gracias", "patterns": ["Gracias", "Muchas gracias"], "responses": ["De nada", "¡Con gusto!"]}
]
}

Paso 4: Procesar los datos

In [5]:
lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_words = ['?', '!']
for intent in intents['intents']:
    for pattern in intent['patterns']: # Added indentation for this line
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print(words)
print(classes)

['adiós', 'buenos', 'días', 'estás', 'gracias', 'hasta', 'hola', 'luego', 'muchas', 'no', 'vemos', '¿cómo']
['despedida', 'gracias', 'saludo']


Paso 5: Crear el modelo

In [7]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = [] # Indented this line to be part of the for loop
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

Paso 6: Crear funciones para el chatbot


In [26]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
              bag[i] = 1
    return(np.array(bag))

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Assuming you have trained a model and it's stored in the variable 'trained_model'
model = training

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = get_response(ints, intents)
    return res

Paso 7: Interactuar con el chatbot

In [28]:

while True:
    message = input("Tú: ")
    if message.lower() == "salir":
        break  # Indent the 'break' statement to be part of the 'if' block
    response = chatbot_response(message)
    print("Chatbot:", response)

Tú: hola


AttributeError: 'list' object has no attribute 'predict'